# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [1]:
print("Hello World!")


Hello World!


In [19]:
import pickle 

with open('data/deduplicated_dentists.pkl', 'rb') as f:
    deduplicated_dentists = pickle.load(f)


In [4]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Get the database credentials from environment variables
user = os.getenv("MYSQL_USER")
password = os.getenv("MYSQL_PASSWORD")
host = os.getenv("MYSQL_HOST")
database = os.getenv("MYSQL_DATABASE")

# Create an SQLAlchemy engine for the connection
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

# Query to retrieve latitude and longitude
query = "SELECT latitude, longitude FROM address WHERE location_type = 'ROOFTOP';"

# Load data into a pandas DataFrame
raw_df = pd.read_sql(query, engine)

patient_df = raw_df

# Display the data
patient_df.head()



,latitude,longitude
0,-36.7875,174.610
1,-36.8152,174.635
2,-36.8385,174.627
3,-36.8337,174.604
4,-36.8430,174.612


In [14]:
patient_df.describe()

,latitude,longitude
count,3761.000000,3761.000000
mean,-36.847161,174.619936
std,0.299675,0.170303
min,-45.462700,168.688000
25%,-36.851400,174.604000
50%,-36.833700,174.617000
75%,-36.813800,174.637000
max,-34.993000,178.040000


In [20]:
import folium
from folium.plugins import HeatMap

# Create a map centered around the average location using deduplicated_dentists
latitude_mean = patient_df['latitude'].mean()  # Assuming your heatmap data is in 'df'
longitude_mean = patient_df['longitude'].mean()

m = folium.Map(location=[latitude_mean, longitude_mean], zoom_start=12)

# Prepare heatmap data from 'df' (which is the original dataset for the heatmap)
heat_data = [[row['latitude'], row['longitude']] for index, row in patient_df.iterrows()]

# Add the HeatMap layer with adjusted radius
HeatMap(heat_data, radius=20).add_to(m)

#Add markers for each practice
for dentist in deduplicated_dentists:
    folium.CircleMarker(
        location=[dentist['lat'], dentist['lon']],
        radius=5,  # smaller radius for a subtle marker
        color='lightgray',
        fill=True,
        fill_color='blue',  # subtle fill color
        fill_opacity=0.6,
        popup=dentist['name']
    ).add_to(m)

# Display the map
m


In [17]:
patient_df.shape

(3761, 2)